In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Packages

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img,img_to_array
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report,confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Conv2D,MaxPooling2D,Dropout,Flatten

Dataset

In [ ]:
dataset_path='/content/drive/MyDrive/sem 7/Lab/DL_Lab/Bean_Dataset'

In [ ]:
def load_images(file_path,img_size):
  images=[]
  labels=[]

  class_names=os.listdir(file_path)

  valid_extensions = (".jpg", ".jpeg", ".png", ".bmp")

  for cls in class_names:

    class_path=os.path.join(file_path,cls)

    for file_name in os.listdir(class_path):

      if file_name.lower().endswith(valid_extensions):

        img=load_img(os.path.join(class_path,file_name),target_size=img_size)
        img_array=img_to_array(img)/255.0
        images.append(img_array)
        labels.append(cls)


  return np.array(images),np.array(labels),class_names


images,labels,cls_names=load_images(dataset_path,(128,128))

In [ ]:
X=images.copy()
y=labels.copy()

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

#### **DNN**

In [ ]:
X_train_flat=X_train.reshape(X_train.shape[0],-1)
X_test_flat=X_test.reshape(X_test.shape[0],-1)

In [ ]:
le=LabelEncoder()
y_train=le.fit_transform(y_train)
y_test=le.transform(y_test)

enc_cls=le.classes_
print(enc_cls)

num_classes=len(enc_cls)
y_train = to_categorical(y_train, num_classes)
y_test  = to_categorical(y_test, num_classes)

In [ ]:
model=Sequential([
    Dense(128,activation='relu',input_shape=(X_train_flat.shape[1],)),
    Dense(64,activation='relu'),
    Dense(len(enc_cls),activation='softmax')
])

In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=["accuracy"])
history1=model.fit(X_train_flat,y_train,epochs=5,validation_split=0.2)

In [ ]:
loss,acc=model.evaluate(X_test_flat,y_test)
print(f'Test Accuracy : {acc*100:.2f}%')
print(f'Test Loss : {loss:.2f}')

In [ ]:
y_pred=np.argmax(model.predict(X_test_flat),axis=-1)
y_true=np.argmax(y_test,axis=-1)

In [ ]:
print(enc_cls)

In [ ]:
print('Classfication Report')
print(classification_report(y_true,y_pred,target_names=enc_cls))

In [ ]:
print('Confusion Matrix')
print(confusion_matrix(y_true,y_pred))

#### **CNN**

In [ ]:
model=Sequential([
    Conv2D(32,(3,3),activation='relu',input_shape=(128,128,3)),
    MaxPooling2D((2,2)),
    Conv2D(64,(3,3),activation='relu'),
    MaxPooling2D((2,2)),
    Flatten(),
    Dense(128,activation='relu'),
    Dropout(0.5),
    Dense(len(cls_names),activation='softmax')
])

In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
history2=model.fit(X_train,y_train,epochs=5,validation_split=0.2)

In [ ]:
loss,acc=model.evaluate(X_test,y_test)
print(f'Test Accuracy : {acc*100:.2f}%')
print(f'Test Loss : {loss:.2f}')

In [ ]:
y_pred=np.argmax(model.predict(X_test),axis=-1)
y_true=np.argmax(y_test,axis=-1)

In [ ]:
print('Classfication Report')
print(classification_report(y_true,y_pred,target_names=enc_cls))

In [ ]:
print('Confusion Matrix')
print(confusion_matrix(y_true,y_pred))

#### **Comparison**

In [ ]:
plt.plot(history1.history['accuracy'],label='train_accuracy')
plt.plot(history1.history['val_accuracy'],label='val_accuracy')
plt.title('Model Accuracy of DNN')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
plt.plot(history2.history['accuracy'],label='train_accuracy')
plt.plot(history2.history['val_accuracy'],label='val_accuracy')
plt.title('Model Accuracy of CNN')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
plt.plot(history1.history['loss'],label='train_loss')
plt.plot(history1.history['val_loss'],label='val_loss')
plt.title('Model loss')
plt.xlabel('Epochs')
plt.ylabel('loss')
plt.legend()
plt.show()

In [ ]:
plt.plot(history2.history['loss'],label='train_loss')
plt.plot(history2.history['val_loss'],label='val_loss')
plt.title('Model loss')
plt.xlabel('Epochs')
plt.ylabel('loss')
plt.legend()
plt.show()